In [ ]:
from urllib.request import urlopen
import pandas as pd
import certifi
import json
from datetime import datetime
import shutil

import yfinance as yf
import os
import pandas_datareader.data as web
from pandas_datareader import wb
import requests

# feature extraction
from talib.abstract import *
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.rcParams['figure.figsize'] = (12,8)
plt.style.use('fivethirtyeight')
%matplotlib inline
import mplfinance as mpf
import seaborn as sn

In [ ]:
'''
Download all the data I need and put them in one csv.
'''

In [ ]:
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.max_columns', 5000)
# pd.set_option('display.width', 1000)

# Download

## Data for certain stcok 

###### yfinance

In [ ]:
# basic daily data
def dl_basic(symbol):
    data = yf.download(symbol, auto_adjust=True, start="2010-01-04", end="2022-06-30")
    data.to_csv(r'./data/'+symbol+'/'+symbol+'.csv',sep=',')

###### data from Financial Modeling Prep api

In [ ]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [ ]:
# three statements, financial ratio and enterprise value 
def dl_financial(symbol):
    name_statement_pair = {'BS':'balance-sheet-statement',
                           'IS':'income-statement',
                           'CF':'cash-flow-statement',
                           'FR':'ratios',
                           'EV':'enterprise-values'
                          }
    for name, statement in name_statement_pair.items():
        url = ('https://financialmodelingprep.com/api/v3/'+statement+'/'+symbol+'?period=quarter&limit=52&apikey=')
        data = get_jsonparsed_data(url)
        with open('./data/'+symbol+'/Financial/'+name+'.json', 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, ensure_ascii=False)

In [ ]:
# number of employees
# Not all companies disclose this number
# def dl_employees(symbol):
#     url = ('https://financialmodelingprep.com/api/v4/historical/employee_count?symbol='+symbol+'&apikey=')
#     data = get_jsonparsed_data(url)
#     Employees = pd.DataFrame({'date':[],
#                              'num_employees':[]})
#     for i in data:
#         Employees = Employees.append(pd.DataFrame({'date':[i['filingDate']],
#              'Employees':[i['employeeCount']]}))

#     Employees.to_csv(r'./data/'+symbol+'/event/Employees.csv',sep=',')

In [ ]:
# ESG json的版本
def dl_ESG(symbol):
    url = ('https://financialmodelingprep.com/api/v4/esg-environmental-social-governance-data?symbol='+symbol+'&apikey=')
    data = get_jsonparsed_data(url)

    with open('./data/'+symbol+'/event/ESG.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False)

In [ ]:
'''peer companies' ohlc and volume, I use peer companies chosen by FMP'''
def dl_peers_basic(symbol):
    url = ('https://financialmodelingprep.com/api/v4/stock_peers?symbol='+symbol+'&apikey=')
    peers = get_jsonparsed_data(url)
    peer_list = peers[0]['peersList']
    for i in peer_list:
        data = yf.download(i, auto_adjust=True, start="2010-01-04", end="2022-06-30")
        data.to_csv(r'./data/'+symbol+'/peers/'+i+'.csv',sep=',')

In [ ]:
# insider trading
def dl_insider_trading(symbol, pages):
    insider_trading = []
    for page in range(pages):
        url = ('https://financialmodelingprep.com/api/v4/insider-trading?symbol='+symbol+'&page='+str(page)+'&apikey=')
        data = get_jsonparsed_data(url)
        for i in range(len(data)):
            if data[i]['transactionType'] == 'P-Purchase' or data[i]['transactionType'] == 'S-Sale':
                insider_trading.append(data[i])
    with open('./data/'+symbol+'/event/Insider_trading.json', 'w', encoding='utf-8') as json_file:
        json.dump(insider_trading, json_file, ensure_ascii=False)

In [ ]:
# senate trading
def dl_senate_trading(symbol):
    url = ('https://financialmodelingprep.com/api/v4/senate-trading?symbol='+symbol+'&apikey=')
    data = get_jsonparsed_data(url)
    with open('./data/'+symbol+'/event/Senate_trading.json', 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False)

###### download all stock data

In [ ]:
def dl_stock_data(symbol, pages):
    if not os.path.isdir(r'./data/'+symbol):
        os.mkdir(r'./data/'+symbol)
    if not os.path.isdir(r'./data/'+symbol+'/event'):
        os.mkdir(r'./data/'+symbol+'/event')
    if not os.path.isdir(r'./data/'+symbol+'/Financial'):
        os.mkdir(r'./data/'+symbol+'/Financial')
    if not os.path.isdir(r'./data/'+symbol+'/peers'):
        os.mkdir(r'./data/'+symbol+'/peers')
    dl_basic(symbol)
    dl_financial(symbol)
#     dl_employees(symbol)
    dl_ESG(symbol)
    dl_peers_basic(symbol)
    dl_insider_trading(symbol, pages)

## General data

In [ ]:
# FF factor
# def df_FF():
#     FFDataTest = pdr.data.DataReader("F-F_Research_Data_5_Factors_2x3_daily", "famafrench", start='2010')[0]
#     FFDataTest.index = pd.to_datetime(FFDataTest.index, format="%Y%m%d", utc=True)
#     FFDataTest.to_csv(r'FFDataTest.csv',sep=',')

###### Alpha Vantage

In [ ]:
# CPI
# def dl_CPI():
#     url = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=Y2IBWUQEKYT0GVR'
#     r = requests.get(url)
#     data = r.json()

#     date_list = []
#     value_list = []
#     for i in data['data']:
#         date_list.append(i['date'])
#         value_list.append(i['value'])
#     CPI = pd.DataFrame({'date':date_list,'CPI':value_list})
#     CPI.to_csv(r'./data/CPI.csv',sep=',')

###### yfinance

In [ ]:
# stock index
# def dl_index():
#     data = yf.download('^VIX', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/VIX.csv',sep=',')
#     data = yf.download('^DJI', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/DJI.csv',sep=',')
#     data = yf.download('^IXIC', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/NASDAQ.csv',sep=',')
#     data = yf.download('^GSPC', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/S&P.csv',sep=',')

In [ ]:
# futures
# def dl_futures():
#     data = yf.download('CL=F', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/OIL_Future.csv',sep=',')
#     data = yf.download('HG=F', start="2010-01-01", end="2022-07-01")
#     data.to_csv(r'./data/COPPER_Future.csv',sep=',')

###### MFP

In [ ]:
# commodities
# def dl_commodities():
#     url = ('https://financialmodelingprep.com/api/v3/historical-price-full/CLUSD?apikey=')
#     data = get_jsonparsed_data(url)
#     with open('./data/Crude_Oil_Commodity.json', 'w', encoding='utf-8') as json_file:
#         json.dump(data, json_file, ensure_ascii=False)
        
#     url = ('https://financialmodelingprep.com/api/v3/historical-price-full/HGUSD?apikey=')
#     data = get_jsonparsed_data(url)
#     with open('./data/Copper_Commodity.json', 'w', encoding='utf-8') as json_file:
#         json.dump(data, json_file, ensure_ascii=False)

## Home made

###### image

In [ ]:
def plot_kandle(symbol):
    # 创建文件夹
    save_path = r'./data/'+symbol+'/images_2'
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.mkdir(save_path)

#     设置mplfinance的蜡烛颜色，up为阳线颜色，down为阴线颜色
#     my_color = mpf.make_marketcolors(up='r',
#                                      down='g',
#                                      edge='inherit',
#                                      wick='inherit',
#                                      volume='inherit')
#     设置图表的背景色
    my_style = mpf.make_mpf_style(figcolor='(0, 0, 0)')

    data = get_basic_data(symbol)

    # 画图并保存
    window = 30
    for i in range(len(data)-window+1):
        df = data[i: i+window]
        start = df[0:1].index[0].strftime('%Y-%m-%d')
        end = df[-1:].index[0].strftime('%Y-%m-%d')
        savefig = r'./data/'+symbol+'/images_2/' + start + '_' + end + '.jpg'
        mpf.plot(df, style=my_style, volume=True, savefig=savefig)

In [ ]:
plot_kandle('CSX')

# Assemble

In [ ]:
def get_basic_data(symbol):
    path = r'./data/'+symbol+'/'+symbol+'.csv'
    data = pd.read_csv(path, index_col=0, parse_dates=True)
    data.index = pd.to_datetime(data.index, format="%Y%m%d")
    return data

In [ ]:
'''
using filling date as the date when the market can have access to the financial report
'''
def get_Fundamental(data, symbol, indicator_list):
    with open('./data/'+symbol+'/Financial/FR.json', 'r', encoding='utf-8') as json_file:
        FR = json.load(json_file)
    with open('./data/'+symbol+'/Financial/BS.json', 'r', encoding='utf-8') as json_file:
        BS = json.load(json_file)
    with open('./data/'+symbol+'/Financial/IS.json', 'r', encoding='utf-8') as json_file:
        IS = json.load(json_file)
    with open('./data/'+symbol+'/Financial/EV.json', 'r', encoding='utf-8') as json_file:
        EV = json.load(json_file)

    for i in FR:
        for j in [BS, IS, EV]:
            for k in j:
                if i['date'] == k['date']:
                    for m,n in k.items():
                        i[m] = n
                        
                        
    # 颠倒顺序是为了填充dataframe方便
    FR.reverse()

    '''
    因为FR中的字典是按照日期从小到大排列的，从FR中选出一组数据i，data中日期大于i，小于i+1的数据就会按照第i组数据的财报来赋值
    '''
    for i in range(len(FR)):
        for j in data.index:
            if i+1 < len(FR):
                if j >= datetime.strptime(FR[i+1]['fillingDate'], "%Y-%m-%d"):
                    continue
            if j >= datetime.strptime(FR[i]['fillingDate'], "%Y-%m-%d"):
                for k in indicator_list:
                    data.loc[j, k] = FR[i][k]
                # PE
                data.loc[j, 'PE'] = data.loc[j, 'Close']/(FR[i]['netIncome']/FR[i]['numberOfShares'])
                # PB
                data.loc[j, 'PB'] = data.loc[j, 'Close']/(FR[i]['revenue']/FR[i]['numberOfShares'])
    return data

In [ ]:
def get_ESG(data, symbol):
    with open('./data/'+symbol+'/event/ESG.json', 'r', encoding='utf-8') as json_file:
        ESG = json.load(json_file)
    ESG.reverse()
    data['ESG'] = 0
    for i in range(len(ESG)):
        for j in data.index:
            if i+1 < len(ESG):
                if j >= datetime.strptime(ESG[i+1]['acceptedDate'][:10], "%Y-%m-%d"):
                    continue
            if j >= datetime.strptime(ESG[i]['acceptedDate'][:10], "%Y-%m-%d"):
                data.loc[j, 'ESG'] = ESG[i]['ESGScore']
    return data

In [ ]:
def get_peers_data(data, symbol):
    for i in os.listdir('./data/'+symbol+'/peers'):
        df = pd.read_csv(r'./data/'+symbol+'/peers/'+i, index_col=0, parse_dates=True)
        df.columns = ['Open','High','Low','P_'+i[:-4], 'Volume']
        df = df['P_'+i[:-4]]
        data = pd.merge(data, df, left_index=True, right_index=True)
    return data

In [ ]:
'''This is super naive, my plan is that:
    when there is a insider purchase someday, the following five day will have a feature of value from 1 to 0.2, and negative value for sales.
    My concern is that maybe sales are not always bad news and vice versa, so I will test another plan in which I only give positive feature
    value no matter what transaction type it is.
    '''
def get_events(data, symbol):
    with open('./data/'+symbol+'/event/Insider_trading.json', 'r', encoding='utf-8') as json_file:
        Insider = json.load(json_file)
    Insider.reverse()
    data['Insider'] = 0
    for i in range(len(Insider)):
        decay = 0
        for j in data.index:
            if i+1 < len(Insider):
                if j >= datetime.strptime(Insider[i+1]['filingDate'][:10], "%Y-%m-%d"):
                    continue
            if j >= datetime.strptime(Insider[i]['filingDate'][:10], "%Y-%m-%d"):
                if Insider[i]['transactionType'] == 'P-Purchase':
                    data.loc[j, 'Insider'] = 5 - decay
                else:
                    data.loc[j, 'Insider'] = -5 + decay
                decay += 1
                if decay == 5:
                    break

    with open('./data/'+symbol+'/event/Senate_trading.json', 'r', encoding='utf-8') as json_file:
        Senate = json.load(json_file)
    Senate.reverse()
    data['Senate'] = 0
    for i in range(len(Senate)):
        decay = 0
        for j in data.index:
            if i+1 < len(Senate):
                if j >= datetime.strptime(Senate[i+1]['dateRecieved'], "%Y-%m-%d"):
                    continue
            if j >= datetime.strptime(Senate[i]['dateRecieved'], "%Y-%m-%d"):
                if Senate[i]['type'] == 'Purchase':
                    data.loc[j, 'Senate'] = 5 - decay
                else:
                    data.loc[j, 'Senate'] = -5 + decay
                decay += 1
                if decay == 5:
                    break
    return data

In [ ]:
def get_general_data(data):
    data.index = pd.to_datetime(data.index, format="%Y%m%d", utc=True)
    # FF factors
    FFDataTest = pd.read_csv(r'data/FFDataTest.csv', index_col=0, parse_dates=True)
    data = pd.merge(data, FFDataTest, left_index=True, right_index=True)
    data = data.drop(['RF'], axis=1)

    FFR = pd.read_csv(r'data/Federal_funds_rate.csv', index_col=1, parse_dates=True).drop(['Unnamed: 0'], axis=1).sort_index()
    FFR.index = pd.to_datetime(FFR.index, format="%Y%m%d", utc=True)
    data = pd.merge(data, FFR, left_index=True, right_index=True)

    CPI = pd.read_csv(r'data/CPI.csv', index_col=1, parse_dates=True).drop(['Unnamed: 0'], axis=1).sort_index()
    CPI = CPI.loc['2010-01-01':]
    CPI.index = pd.to_datetime(CPI.index, format="%Y%m%d", utc=True)
    for i in CPI.index:
        for j in data.index:
            if str(i)[:7] == str(j)[:7]:
                data.loc[j, 'CPI'] = CPI.loc[i][0]

    data_list = ['DJI', 'NASDAQ', 'S&P', 'COPPER_Future', 'OIL_Future', 'VIX']
    for d in data_list:
        df = pd.read_csv(r'data/'+d+'.csv', index_col=0, parse_dates=True)
        df.drop(['Close'], axis=1, inplace=True)
        df.columns = [d+'_'+i for i in df.columns]
        df.index = pd.to_datetime(df.index, format="%Y%m%d", utc=True)
        data = pd.merge(data, df, left_index=True, right_index=True)

    data.drop(['VIX_Volume'], axis=1, inplace=True)
    
#     with open('./data/Crude_Oil_Commodity.json', 'r', encoding='utf-8') as json_file:
#         Crude_Oil_Commodity = json.load(json_file)
#     df = pd.DataFrame({'date':[],
#                      'Crude_Oil_Commodity':[]})
#     for i in Crude_Oil_Commodity['historical']:
#         df = df.append(pd.DataFrame({'date':[i['date']],
#                                      'Crude_Oil_Commodity':[i['adjClose']]}))
#     df = df.set_index('date')
#     df = df.sort_index()
#     df.index = pd.to_datetime(df.index, format="%Y-%m-%d", utc=True)
#     data = pd.merge(data, df, left_index=True, right_index=True)
    
#     with open('./data/Copper_Commodity.json', 'r', encoding='utf-8') as json_file:
#         Copper_Commodity = json.load(json_file)
#     df = pd.DataFrame({'date':[],
#                      'Copper_Commodity':[]})
#     for i in Copper_Commodity['historical']:
#         df = df.append(pd.DataFrame({'date':[i['date']],
#                                      'Copper_Commodity':[i['adjClose']]}))
#     df = df.set_index('date')
#     df = df.sort_index()
#     df.index = pd.to_datetime(df.index, format="%Y-%m-%d", utc=True)
#     data = pd.merge(data, df, left_index=True, right_index=True)
    
    return data

In [ ]:
def add_features(data):
    df = data.copy()
    
    # Tecnical Indicators
    window_list = [5, 10, 21, 50, 200]
    for i in window_list:
        df[f'MA_{i}'] = MA(df['Close'], i)
        df[f'SMA_{i}'] = SMA(df['Close'], i)
        df[f'EMA_{i}'] = EMA(df['Close'], i)
    df['RSI_5'] = RSI(df['Close'], 5)
    df['RSI_14'] = RSI(df['Close'], 14)
    df['ATR_14'] = ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['CCI_14'] = CCI(df['High'], df['Low'], df['Close'], timeperiod=14)
    df['CCI_24'] = CCI(df['High'], df['Low'], df['Close'], timeperiod=24)
    df['BBANDS_5_UB'],df['BBANDS_5_MB'],df['BBANDS_5_LB'] = BBANDS(df['Close'], timeperiod=5)
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['Slow_K'], df['Slow_D'] = STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    df['ADOSC'] = ADOSC(df['High'], df['Low'], df['Close'], df['Volume'], fastperiod=3, slowperiod=10)

    # 计算n天的YZ方差
    n = 10
    df['C_0'] = df['Close'].shift()     # 昨天的收盘价
    df['o'] = np.log(df['Open']/df['C_0'])
    df['u'] = np.log(df['High']/df['Open'])
    df['d'] = np.log(df['Low']/df['Open'])
    df['c'] = np.log(df['Close']/df['Open'])
    # 计算V_RS
    df['V_RS'] = (df['u']*(df['u']-df['c'])+df['d']*(df['d']-df['c'])).rolling(n).mean()
    # 计算V_o
    df['V_o'] = (df['o'] - df['o'].rolling(n).mean()).rolling(n).var()
    # 计算V_c
    df['V_c'] = (df['c'] - df['c'].rolling(n).mean()).rolling(n).var()
    # 计算V_YZ
    k = 0.34/(1.34+(n+1)/(n-1))
    df['V_YZ'] = df['V_o'] + k*df['V_c'] + (1-k)*df['V_RS']
    df = df.drop(['V_RS', 'V_o', 'V_c', 'o', 'u', 'd', 'c', 'C_0'], axis=1)

    # Fluctuation Percentage
    fluc_per_list = ['SMA_10', 'RSI_5', 'Slow_K', 'Slow_D']
    for i in fluc_per_list:    
        scaler = MinMaxScaler(feature_range=(-1, 1))
        df[i+'_fluc_per'] = scaler.fit_transform(df[i].pct_change().values.reshape(-1,1))

    # Polarize
    pol_list = ['MACD', 'MACD_Signal', 'MACD_Hist', 'CCI_24', 'ADOSC']
    for i in pol_list:
        df[i+'_pol'] = np.where(df[i]>0, 1, -1)

    # Min-Max
    MM_list = ['Volume', 'SMA_10', 'RSI_5', 'Slow_K', 'Slow_D', 'ADOSC']
    for i in MM_list:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        df[i+'_MM'] = scaler.fit_transform(df[i].values.reshape(-1,1))

    # Other Features
    window=20
    df.dropna(inplace=True)
    df['r'] = np.log(df['Close'] / df['Close'].shift())
    df['Min_20'] = df['Close'].rolling(window).min()
    df['Max_20'] = df['Close'].rolling(window).max()
    df['Std_20'] = df['r'].rolling(window).std()
    df = df.drop('r', axis=1)
    
    df.dropna(inplace=True)
    
    return df

In [ ]:
def Avengers_Assemble(symbol):
    data = get_basic_data(symbol)
    indicator_list = ['cashRatio', 'quickRatio', 'currentRatio', 'debtEquityRatio', 'receivablesTurnover', 'inventoryTurnover'
                  ,'grossProfitMargin', 'netProfitMargin', 'returnOnEquity', 'dividendPayoutRatio']
    data = get_Fundamental(data, symbol, indicator_list)
    data = get_ESG(data,symbol)
    data = get_peers_data(data, symbol)
    data = get_events(data, symbol)
    data = get_general_data(data)
    data = add_features(data)

    data.to_csv(r'./data/'+symbol+'/'+symbol+'_ready_to_use.csv', sep=',')

In [ ]:
Avengers_Assemble('AAPL')

In [ ]:
data = pd.read_csv(r'./data/CSX/CSX_ready_to_use.csv', index_col=0, parse_dates=True)